In [45]:
import pandas as pd
import numpy as np
import re
import datetime

#get raw data 
dir_path='//Users/rony/Dropbox/2. projects/AMR/Analysis/deprivation/IMD_2015/raw_data/File_13_ID_2015_Clinical_Commissioning_Group_Summaries.xlsx'
xl = pd.ExcelFile(dir_path)
sheets=xl.sheet_names
del sheets[0]
#storing the values 'Rank of average score' of each index in a separate datdframes
indices_dfs=[] #to store dataframes for indices datasets 
data=pd.DataFrame()
for index, value in enumerate (sheets):
    sheet_df=xl.parse(value)
    columns_list=sheet_df.columns
    for index1, header in enumerate (columns_list):
        searching=re.findall ('Rank of average score', header) 
        if searching ==['Rank of average score']:
            if index==0:
                sheet_df=sheet_df[['Clinical Commissioning Group Name (2015)','Clinical Commissioning Group Code (2015)', header]]
            else:
                sheet_df=sheet_df[['Clinical Commissioning Group Name (2015)',header]]
                
            indices_dfs=indices_dfs+[sheet_df]
            sheet_df.rename(columns = {'Clinical Commissioning Group Name (2015)':'CCG'}, inplace = True)

#join datasets
indices_dfs = [df.set_index('CCG') for df in indices_dfs]
indices_dfs=indices_dfs[0].join(indices_dfs[1:])

#rename headers
headers=indices_dfs.columns
new_headers=[re.findall('(.*) - Rank of average score',x) for x in headers]
new_headers [0]=['AreaCode']
new_headers=[z[0] for z in new_headers]
indices_dfs.columns=new_headers

#clean CCG names
temp_series=indices_dfs.index.str.rstrip('CCG')
temp_series1=temp_series.str.lstrip('NHS')
temp_series2=temp_series1.str.strip()
indices_dfs.index=temp_series2

#import and join descriptive data
descriptives=pd.read_csv('diff_desc_92377.csv')
descriptives=descriptives[['CCG','AreaCode','mean','max','diff.','%diff.']]

#match CCG names in both datasets
#necessary to examine non-overlapping CCGs (see regions_not_included.txt)
descriptives['CCG']=descriptives['CCG'].str.replace('Of', 'of')
descriptives['CCG']=descriptives['CCG'].str.replace('And', 'and')

descriptives['CCG']=descriptives['CCG'].str.replace('Airedale, Wharfdale and Craven','Airedale, Wharfedale and Craven')
descriptives['CCG']=descriptives['CCG'].str.replace('Birmingham Crosscity', 'Birmingham CrossCity')
descriptives['CCG']=descriptives['CCG'].str.replace('Blackburn With Darwen','Blackburn with Darwen')
descriptives['CCG']=descriptives['CCG'].str.replace('Hartlepool and Stockton-On-Tees','Hartlepool and Stockton-on-Tees')
descriptives['CCG']=descriptives['CCG'].str.replace('Newcastle and Gateshead', 'Newcastle Gateshead')
descriptives['CCG']=descriptives['CCG'].str.replace('South East Staffs and Seisdon Peninsular','South East Staffordshire and Seisdon Peninsular')
descriptives['CCG']=descriptives['CCG'].str.replace('Stoke On Trent','Stoke on Trent')

desc_IMD=pd.merge (descriptives,indices_dfs)
#rename columns
desc_IMD=desc_IMD.rename(columns={'Education, Skills and Training':'Education', 'Health Deprivation and Disability':'Health',
       'Barriers to Housing and Services':'Services_Housing','Living Environment':'Living_Environment'})
del desc_IMD['Income Deprivation Affecting Children Index (IDACI)'], desc_IMD['Income Deprivation Affecting Older People (IDAOPI)']
desc_IMD=desc_IMD.rename(columns={'mean': 'mean_presc','IMD':'Deprivation', 'max': 'max_presc','%diff.':'Reduction'})

desc_IMD.to_csv('desc_IMD_92377.csv')